In [17]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import KeyedVectors
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB  # Import Naive Bayes
from sklearn.ensemble import StackingClassifier

# Load the dataset
train_data = pd.read_csv('/kaggle/input/lemma-balanced/lemmatized_dataset_final_balanced_train.csv')
valid_data = pd.read_csv('/kaggle/input/lemma-balanced/lemmatized_dataset_final_balanced_validation.csv')
test_data = pd.read_csv('/kaggle/input/lemma-balanced/lemmatized_dataset_final_balanced_test.csv')

# Display the first few rows of the dataset
train_data.head()

,Headline,Body ID,Stance,articleBody,stance_cat
0,dna test confirm lebanon is holding isi leader...,2042,unrelated,there is a story currently making the round ab...,3
1,somalia shebab chief ahmed abdi godane likely ...,1610,discuss,ahmed abdi godane the leader of al shabab the ...,2
2,dna test prove lebanon is holding isi chief al...,1468,disagree,an iraqi official denied that a woman detained...,1
3,the pumpkinspice condom is just a figment of y...,1253,unrelated,the united state department of defense said on...,3
4,u probing claim isi fighter seized airdropped ...,465,discuss,the pentagon admitted on wednesday that isi di...,2


In [18]:
valid_data.head()

,Headline,Body ID,Stance,articleBody,stance_cat
0,james foley american journalist james wright f...,1336,agree,two yous official say they believe american jo...,0
1,yous hostage luke somers killed during yemen r...,1119,discuss,a britishborn photographer held hostage by al ...,2
2,the internet tried to make axl rose it latest ...,1857,unrelated,those caught with the same moniker a the dicta...,3
3,conde nast rat problem at one world trade cent...,821,agree,the rat infestation in vogue new 1 world trade...,0
4,video marine survives taliban sniper headshot ...,1256,unrelated,the pentagon ha confirmed that ahmed abdi goda...,3


In [19]:
test_data.head()

,Headline,Body ID,Stance,articleBody,stance_cat
0,god is a woman priest who died for 48 minute c...,1227,disagree,a supposed catholic priest claim of seeing god...,1
1,update internet report of ebola outbreak in pu...,958,unrelated,an animal lover from norfolk splashed hundred ...,3
2,british hostage david haines beheaded by islam...,2034,unrelated,when a report went viral that nbc meteorologis...,3
3,video messaging service verifies timing of cnn...,1435,unrelated,apple may be planning to hold a special event ...,3
4,omar gonzalez white house fencejumper made it ...,917,unrelated,source detroit free press yesterday we reporte...,3


In [20]:
train_data['stance_cat'] = train_data['Stance'].map({'agree':0,'disagree':1,'discuss':2,'unrelated':3}).astype(int)
train_data['Stance'].value_counts()

Stance
unrelated    10742
discuss       7127
agree         2942
disagree       672
Name: count, dtype: int64

In [21]:
valid_data['stance_cat'] = valid_data['Stance'].map({'agree':0,'disagree':1,'discuss':2,'unrelated':3}).astype(int)
valid_data['Stance'].value_counts()

Stance
unrelated    1343
discuss       891
agree         368
disagree       84
Name: count, dtype: int64

In [22]:
test_data['stance_cat'] = test_data['Stance'].map({'agree':0,'disagree':1,'discuss':2,'unrelated':3}).astype(int)
test_data['Stance'].value_counts()

Stance
unrelated    1342
discuss       891
agree         368
disagree       84
Name: count, dtype: int64

In [23]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print('CUDA is available. GPU will be used.')
else:
    print('CUDA is not available. CPU will be used.')

CUDA is available. GPU will be used.


In [24]:
# Updated preprocess_text function
def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    return " ".join(tokens)

# Applying the updated preprocess_text function
train_data['headline_processed'] = train_data['Headline'].apply(preprocess_text)
train_data['body_processed'] = train_data['articleBody'].apply(preprocess_text)

valid_data['headline_processed'] = valid_data['Headline'].apply(preprocess_text)
valid_data['body_processed'] = valid_data['articleBody'].apply(preprocess_text)

test_data['headline_processed'] = test_data['Headline'].apply(preprocess_text)
test_data['body_processed'] = test_data['articleBody'].apply(preprocess_text)

In [25]:
# Count how many times each gram appears in the headline and body
headline_unigrams = train_data['headline_processed'].apply(lambda x: x.split())
body_unigrams = train_data['body_processed'].apply(lambda x: x.split())

# Count the number of unique grams in the headline and body
headline_unique_counts = headline_unigrams.apply(lambda x: len(set(x)))
body_unique_counts = body_unigrams.apply(lambda x: len(set(x)))

# Compute the ratio between the count of grams in the headline and body
ratio_counts = headline_unique_counts / body_unique_counts

# Compute overlapping count of grams between headline and body
overlapping_counts = []
for headline, body in zip(headline_unigrams, body_unigrams):
    overlapping_counts.append(len(set(headline) & set(body)))

# Normalize overlapping count by the number of grams in the headline
normalized_overlapping_counts = [oc / len(h) if len(h) > 0 else 0 for oc, h in zip(overlapping_counts, headline_unigrams)]

# Create DataFrame for count features
count_features_train = pd.DataFrame({
    'headline_unique_counts': headline_unique_counts,
    'body_unique_counts': body_unique_counts,
    'ratio_counts': ratio_counts,
    'overlapping_counts': overlapping_counts,
    'normalized_overlapping_counts': normalized_overlapping_counts
})

#Validation set 
# Count how many times each gram appears in the headline and body
headline_unigrams = valid_data['headline_processed'].apply(lambda x: x.split())
body_unigrams = valid_data['body_processed'].apply(lambda x: x.split())

# Count the number of unique grams in the headline and body
headline_unique_counts = headline_unigrams.apply(lambda x: len(set(x)))
body_unique_counts = body_unigrams.apply(lambda x: len(set(x)))

# Compute the ratio between the count of grams in the headline and body
ratio_counts = headline_unique_counts / body_unique_counts

# Compute overlapping count of grams between headline and body
overlapping_counts = []
for headline, body in zip(headline_unigrams, body_unigrams):
    overlapping_counts.append(len(set(headline) & set(body)))

# Normalize overlapping count by the number of grams in the headline
normalized_overlapping_counts = [oc / len(h) if len(h) > 0 else 0 for oc, h in zip(overlapping_counts, headline_unigrams)]

# Create DataFrame for count features
count_features_valid = pd.DataFrame({
    'headline_unique_counts': headline_unique_counts,
    'body_unique_counts': body_unique_counts,
    'ratio_counts': ratio_counts,
    'overlapping_counts': overlapping_counts,
    'normalized_overlapping_counts': normalized_overlapping_counts
})

# Applying the same process for test data
# Count how many times each gram appears in the headline and body
test_headline_unigrams = test_data['headline_processed'].apply(lambda x: x.split())
test_body_unigrams = test_data['body_processed'].apply(lambda x: x.split())

# Count the number of unique grams in the headline and body
test_headline_unique_counts = test_headline_unigrams.apply(lambda x: len(set(x)))
test_body_unique_counts = test_body_unigrams.apply(lambda x: len(set(x)))

# Compute the ratio between the count of grams in the headline and body
test_ratio_counts = test_headline_unique_counts / test_body_unique_counts

# Compute overlapping count of grams between headline and body
test_overlapping_counts = []
for headline, body in zip(test_headline_unigrams, test_body_unigrams):
    test_overlapping_counts.append(len(set(headline) & set(body)))

# Normalize overlapping count by the number of grams in the headline
test_normalized_overlapping_counts = [oc / len(h) if len(h) > 0 else 0 for oc, h in zip(test_overlapping_counts, test_headline_unigrams)]

# Create DataFrame for count features for test data
count_features_test = pd.DataFrame({
    'headline_unique_counts': test_headline_unique_counts,
    'body_unique_counts': test_body_unique_counts,
    'ratio_counts': test_ratio_counts,
    'overlapping_counts': test_overlapping_counts,
    'normalized_overlapping_counts': test_normalized_overlapping_counts
})

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# TF-IDF Vectorization for training data
headline_tfidf_train = tfidf_vectorizer.fit_transform(train_data['headline_processed'])
body_tfidf_train = tfidf_vectorizer.transform(train_data['body_processed'])

# Compute cosine similarity between headline and body vectors for training data
cosine_similarities_train = []
for headline_vec, body_vec in zip(headline_tfidf_train, body_tfidf_train):
    cosine_similarities_train.append(cosine_similarity(headline_vec.reshape(1, -1), body_vec.reshape(1, -1))[0][0])

# Create DataFrame for TF-IDF features for training data
tfidf_features_train = pd.DataFrame({
    'cosine_similarity': cosine_similarities_train
})

headline_tfidf_valid = tfidf_vectorizer.fit_transform(valid_data['headline_processed'])
body_tfidf_valid = tfidf_vectorizer.transform(valid_data['body_processed'])

# Compute cosine similarity between headline and body vectors for training data
cosine_similarities_valid = []
for headline_vec, body_vec in zip(headline_tfidf_valid, body_tfidf_valid):
    cosine_similarities_valid.append(cosine_similarity(headline_vec.reshape(1, -1), body_vec.reshape(1, -1))[0][0])

# Create DataFrame for TF-IDF features for training data
tfidf_features_valid = pd.DataFrame({
    'cosine_similarity': cosine_similarities_valid
})


# TF-IDF Vectorization for testing data
headline_tfidf_test = tfidf_vectorizer.transform(test_data['headline_processed'])
body_tfidf_test = tfidf_vectorizer.transform(test_data['body_processed'])

# Compute cosine similarity between headline and body vectors for testing data
cosine_similarities_test = []
for headline_vec, body_vec in zip(headline_tfidf_test, body_tfidf_test):
    cosine_similarities_test.append(cosine_similarity(headline_vec.reshape(1, -1), body_vec.reshape(1, -1))[0][0])

# Create DataFrame for TF-IDF features for testing data
tfidf_features_test = pd.DataFrame({
    'cosine_similarity': cosine_similarities_test
})

In [27]:
from sklearn.decomposition import TruncatedSVD

# Perform Singular Value Decomposition (SVD) on TF-IDF features for train dataset
svd_train = TruncatedSVD(n_components=100)  # Adjust the number of components as needed
headline_svd_train = svd_train.fit_transform(headline_tfidf_train)
body_svd_train = svd_train.transform(body_tfidf_train)

# Compute cosine similarity between SVD features of headline and body for train dataset
cosine_sim_svd_train = []
for h_svd, b_svd in zip(headline_svd_train, body_svd_train):
    cosine_sim_svd_train.append(cosine_similarity(h_svd.reshape(1, -1), b_svd.reshape(1, -1))[0][0])

# Create DataFrame for SVD features for train dataset
svd_features_train = pd.DataFrame({
    'cosine_similarity_svd': cosine_sim_svd_train
})

# Perform Singular Value Decomposition (SVD) on TF-IDF features for train dataset
svd_valid = TruncatedSVD(n_components=100)  # Adjust the number of components as needed
headline_svd_valid = svd_valid.fit_transform(headline_tfidf_valid)
body_svd_valid = svd_valid.transform(body_tfidf_valid)

# Compute cosine similarity between SVD features of headline and body for train dataset
cosine_sim_svd_valid = []
for h_svd, b_svd in zip(headline_svd_valid, body_svd_valid):
    cosine_sim_svd_valid.append(cosine_similarity(h_svd.reshape(1, -1), b_svd.reshape(1, -1))[0][0])

# Create DataFrame for SVD features for train dataset
svd_features_valid = pd.DataFrame({
    'cosine_similarity_svd': cosine_sim_svd_valid
})

# Perform Singular Value Decomposition (SVD) on TF-IDF features for test dataset
svd_test = TruncatedSVD(n_components=100)  # Adjust the number of components as needed
headline_svd_test = svd_test.fit_transform(headline_tfidf_test)
body_svd_test = svd_test.transform(body_tfidf_test)

# Compute cosine similarity between SVD features of headline and body for test dataset
cosine_sim_svd_test = []
for h_svd, b_svd in zip(headline_svd_test, body_svd_test):
    cosine_sim_svd_test.append(cosine_similarity(h_svd.reshape(1, -1), b_svd.reshape(1, -1))[0][0])

# Create DataFrame for SVD features for test dataset
svd_features_test = pd.DataFrame({
    'cosine_similarity_svd': cosine_sim_svd_test
})

In [28]:
# Install Kaggle API
!pip install kaggle

# Set up Kaggle API credentials (replace 'username' and 'key' with your Kaggle username and API key)
!mkdir ~/.kaggle
!echo '{"username":"joshidevanshi","key":"4e038483c682fdbfb691f4fb95f5a416"}' > ~/.kaggle/kaggle.json

# Download the Google News Word2Vec embeddings dataset
!kaggle datasets download -d leadbest/googlenewsvectorsnegative300

# Unzip the downloaded file
!unzip googlenewsvectorsnegative300.zip

mkdir: cannot create directory '/root/.kaggle': File exists
googlenewsvectorsnegative300.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  googlenewsvectorsnegative300.zip
replace GoogleNews-vectors-negative300.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [30]:
from gensim.models import KeyedVectors
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained word vectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Function to generate word vectors for a given text
def generate_word_vectors(text):
    vectors = []
    tokens = text.split()
    for token in tokens:
        if token in word_vectors:
            vectors.append(word_vectors[token])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word_vectors.vector_size) 

# Generate word vectors for headline and body for train dataset
headline_word2vec_train = train_data['headline_processed'].apply(generate_word_vectors)
body_word2vec_train = train_data['body_processed'].apply(generate_word_vectors)

# Compute cosine similarity between Word2Vec features of headline and body for train dataset
cosine_sim_word2vec_train = []
for h_w2v, b_w2v in zip(headline_word2vec_train, body_word2vec_train):
    cosine_sim_word2vec_train.append(cosine_similarity(h_w2v.reshape(1, -1), b_w2v.reshape(1, -1))[0][0])

# Create DataFrame for Word2Vec features for train dataset
word2vec_features_train = pd.DataFrame({
    'cosine_similarity_word2vec': cosine_sim_word2vec_train
})

# Generate word vectors for headline and body for train dataset
headline_word2vec_valid = valid_data['headline_processed'].apply(generate_word_vectors)
body_word2vec_valid = valid_data['body_processed'].apply(generate_word_vectors)

# Compute cosine similarity between Word2Vec features of headline and body for train dataset
cosine_sim_word2vec_valid = []
for h_w2v, b_w2v in zip(headline_word2vec_valid, body_word2vec_valid):
    cosine_sim_word2vec_valid.append(cosine_similarity(h_w2v.reshape(1, -1), b_w2v.reshape(1, -1))[0][0])

# Create DataFrame for Word2Vec features for train dataset
word2vec_features_valid = pd.DataFrame({
    'cosine_similarity_word2vec': cosine_sim_word2vec_valid
})

# Generate word vectors for headline and body for test dataset
headline_word2vec_test = test_data['headline_processed'].apply(generate_word_vectors)
body_word2vec_test = test_data['body_processed'].apply(generate_word_vectors)

# Compute cosine similarity between Word2Vec features of headline and body for test dataset
cosine_sim_word2vec_test = []
for h_w2v, b_w2v in zip(headline_word2vec_test, body_word2vec_test):
    cosine_sim_word2vec_test.append(cosine_similarity(h_w2v.reshape(1, -1), b_w2v.reshape(1, -1))[0][0])

# Create DataFrame for Word2Vec features for test dataset
word2vec_features_test = pd.DataFrame({
    'cosine_similarity_word2vec': cosine_sim_word2vec_test
})

In [31]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Function to calculate sentiment polarity scores for train dataset
def calculate_sentiment_train(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']  # Using compound score as it combines positive, negative, and neutral scores

# Calculate sentiment polarity scores for headline and body for train dataset
headline_sentiment_train = train_data['headline_processed'].apply(calculate_sentiment_train)
body_sentiment_train = train_data['body_processed'].apply(calculate_sentiment_train)

# Create DataFrame for sentiment features for train dataset
sentiment_features_train = pd.DataFrame({
    'headline_sentiment': headline_sentiment_train,
    'body_sentiment': body_sentiment_train
})

def calculate_sentiment_valid(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']  # Using compound score as it combines positive, negative, and neutral scores

# Calculate sentiment polarity scores for headline and body for train dataset
headline_sentiment_valid = valid_data['headline_processed'].apply(calculate_sentiment_valid)
body_sentiment_valid = valid_data['body_processed'].apply(calculate_sentiment_valid)

# Create DataFrame for sentiment features for train dataset
sentiment_features_valid = pd.DataFrame({
    'headline_sentiment': headline_sentiment_valid,
    'body_sentiment': body_sentiment_valid
})

# Function to calculate sentiment polarity scores for test dataset
def calculate_sentiment_test(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']  # Using compound score as it combines positive, negative, and neutral scores

# Calculate sentiment polarity scores for headline and body for test dataset
headline_sentiment_test = test_data['headline_processed'].apply(calculate_sentiment_test)
body_sentiment_test = test_data['body_processed'].apply(calculate_sentiment_test)

# Create DataFrame for sentiment features for test dataset
sentiment_features_test = pd.DataFrame({
    'headline_sentiment': headline_sentiment_test,
    'body_sentiment': body_sentiment_test
})

In [32]:
from sklearn.metrics import accuracy_score

# Concatenate all the generated features for train dataset
all_features_train = pd.concat([count_features_train, tfidf_features_train, svd_features_train, word2vec_features_train, sentiment_features_train], axis=1)

# Define features and labels for train dataset
X_train = all_features_train
y_train = train_data['stance_cat']

# Construct LightGBM classifier
base_model = lgb.LGBMClassifier()
final_model = GaussianNB()

clf = StackingClassifier(
    estimators=[('lgb', base_model)], 
    final_estimator=final_model,
    stack_method='predict_proba', 
    passthrough=False  
)

# Fit classifier on train dataset
clf.fit(X_train, y_train)

# Perform cross-validation on train dataset
scores_train = cross_val_score(clf, X_train, y_train, cv=10)
print("Accuracy for Train Dataset:", np.mean(scores_train))

# Concatenate all the generated features for validation dataset
all_features_valid = pd.concat([count_features_valid, tfidf_features_valid, svd_features_valid, word2vec_features_valid, sentiment_features_valid], axis=1)

# Define features and labels for validation dataset
X_valid = all_features_valid
y_valid = valid_data['stance_cat']

# Predict on validation dataset
y_pred_valid = clf.predict(X_valid)

# Calculate accuracy on validation dataset
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
print("Accuracy for Validation Dataset:", accuracy_valid)

# Concatenate all the generated features for test dataset
all_features_test = pd.concat([count_features_test, tfidf_features_test, svd_features_test, word2vec_features_test, sentiment_features_test], axis=1)

# Define features and labels for test dataset
X_test = all_features_test
y_test = test_data['stance_cat']

# Predict on test dataset
y_pred_test = clf.predict(X_test)

# Calculate accuracy on test dataset
accuracy_test = accuracy_score(y_test, y_pred_test)
print("Accuracy for Test Dataset:", accuracy_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1963
[LightGBM] [Info] Number of data points in the train set: 21483, number of used features: 10
[LightGBM] [Info] Start training from score -1.988172
[LightGBM] [Info] Start training from score -3.464759
[LightGBM] [Info] Start training from score -1.103372
[LightGBM] [Info] Start training from score -0.693101
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001584 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1948
[LightGBM] [Info] Number of data points in the train set: 17186, number of used features: 10
[LightGBM] [Info] Start training from score -1.987979
[LightGBM] [Info] Start training from score -3.465852
[LightGBM] [Info] Start training from score -1.103278
[LightGBM] [Info] Start training from score 

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Train the LightGBoost classifier on train dataset
clf.fit(X_train, y_train)

# Make predictions on train dataset
y_train_pred = clf.predict(X_train)

# Calculate precision, recall, and F1-score for each class on train dataset
precision_train = precision_score(y_train, y_train_pred, average=None)
recall_train = recall_score(y_train, y_train_pred, average=None)
f1_train = f1_score(y_train, y_train_pred, average=None)

# Calculate macro-averaged precision, recall, and F1-score on train dataset
macro_precision_train = precision_score(y_train, y_train_pred, average='macro')
macro_recall_train = recall_score(y_train, y_train_pred, average='macro')
macro_f1_train = f1_score(y_train, y_train_pred, average='macro')

# Print the results for train dataset
print("Train Dataset:")
print("LightGBoost Accuracy:", np.mean(scores_train))
print("Class 0 - Precision:", precision_train[0], ", Recall:", recall_train[0], ", F1-score:", f1_train[0])
print("Class 1 - Precision:", precision_train[1], ", Recall:", recall_train[1], ", F1-score:", f1_train[1])
print("Class 2 - Precision:", precision_train[2], ", Recall:", recall_train[2], ", F1-score:", f1_train[2])
print("Class 3 - Precision:", precision_train[3], ", Recall:", recall_train[3], ", F1-score:", f1_train[3])
print("Macro Precision:", macro_precision_train)
print("Macro Recall:", macro_recall_train)
print("Macro F1 Score:", macro_f1_train)

# Make predictions on test dataset
y_valid_pred = clf.predict(X_valid)

# Calculate precision, recall, and F1-score for each class on test dataset
precision_valid = precision_score(y_valid, y_valid_pred, average=None)
recall_valid = recall_score(y_valid, y_valid_pred, average=None)
f1_valid = f1_score(y_valid, y_valid_pred, average=None)

# Calculate macro-averaged precision, recall, and F1-score on test dataset
macro_precision_valid = precision_score(y_valid, y_valid_pred, average='macro')
macro_recall_valid = recall_score(y_valid, y_valid_pred, average='macro')
macro_f1_valid = f1_score(y_valid, y_valid_pred, average='macro')

# Print the results for test dataset
print("\nValidation Dataset:")
print("Accuracy for Validation Dataset:", accuracy_valid)
print("Class 0 - Precision:", precision_valid[0], ", Recall:", recall_valid[0], ", F1-score:", f1_valid[0])
print("Class 1 - Precision:", precision_valid[1], ", Recall:", recall_valid[1], ", F1-score:", f1_valid[1])
print("Class 2 - Precision:", precision_valid[2], ", Recall:", recall_valid[2], ", F1-score:", f1_valid[2])
print("Class 3 - Precision:", precision_valid[3], ", Recall:", recall_valid[3], ", F1-score:", f1_valid[3])
print("Macro Precision:", macro_precision_valid)
print("Macro Recall:", macro_recall_valid)
print("Macro F1 Score:", macro_f1_valid)



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002664 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1963
[LightGBM] [Info] Number of data points in the train set: 21483, number of used features: 10
[LightGBM] [Info] Start training from score -1.988172
[LightGBM] [Info] Start training from score -3.464759
[LightGBM] [Info] Start training from score -1.103372
[LightGBM] [Info] Start training from score -0.693101
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001590 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1948
[LightGBM] [Info] Number of data points in the train set: 17186, number of used features: 10
[LightGBM] [Info] Start training from score -1.987979
[LightGBM] [Info] Start training from score -3.465852
[LightGBM] [Info] Start training from score -1.103278
[LightGBM] [Info] Start training from score 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
def score_submission(gold_labels, test_labels):
    score = 0.0

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in [0, 1, 2] and t_stance in [0, 1, 2]:
            score += 0.25

    return score

def report_score(actual,predicted):
    score = score_submission(actual,predicted)
    best_score = score_submission(actual,actual)

    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score

In [46]:
# Make predictions on test dataset
y_test_pred = clf.predict(X_test)

# Calculate precision, recall, and F1-score for each class on test dataset
precision_test = precision_score(y_test, y_test_pred, average=None)
recall_test = recall_score(y_test, y_test_pred, average=None)
f1_test = f1_score(y_test, y_test_pred, average=None)

# Calculate macro-averaged precision, recall, and F1-score on test dataset
macro_precision_test = precision_score(y_test, y_test_pred, average='macro')
macro_recall_test = recall_score(y_test, y_test_pred, average='macro')
macro_f1_test = f1_score(y_test, y_test_pred, average='macro')

# Print the results for test dataset
print("\nTest Dataset:")
print("Accuracy for Test Dataset:", accuracy_test)
print("Class 0 - Precision:", precision_test[0], ", Recall:", recall_test[0], ", F1-score:", f1_test[0])
print("Class 1 - Precision:", precision_test[1], ", Recall:", recall_test[1], ", F1-score:", f1_test[1])
print("Class 2 - Precision:", precision_test[2], ", Recall:", recall_test[2], ", F1-score:", f1_test[2])
print("Class 3 - Precision:", precision_test[3], ", Recall:", recall_test[3], ", F1-score:", f1_test[3])
print("Macro Precision:", macro_precision_test)
print("Macro Recall:", macro_recall_test)
print("Macro F1 Score:", macro_f1_test)

custom_score = score_submission(y_test, y_test_pred)
report_sc = report_score(y_test, y_test_pred)

print("Custom Score:", custom_score)



Test Dataset:
Accuracy for Test Dataset: 0.8249534450651769
Class 0 - Precision: 0.576271186440678 , Recall: 0.46195652173913043 , F1-score: 0.5128205128205129
Class 1 - Precision: 0.3392857142857143 , Recall: 0.2261904761904762 , F1-score: 0.27142857142857146
Class 2 - Precision: 0.7446601941747573 , Recall: 0.8608305274971941 , F1-score: 0.7985424258198854
Class 3 - Precision: 0.9654907975460123 , Recall: 0.9381520119225037 , F1-score: 0.9516250944822373
Macro Precision: 0.6564269731117904
Macro Recall: 0.6217823843373261
Macro F1 Score: 0.6336041511378018
Score: 1985.75 out of 2349.5	(84.51798254947862%)
Custom Score: 1985.75
